In [1]:
import torch as tc
import pandas as pd
from src.bbox import BBox
from src.mesh_manipulation import save_obj
from tqdm import tqdm
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    #'neck_waist': 5
}

In [33]:
subdivided_bodies = tc.load('data/ss20_subdivided_bodies.pt')
gender_curves = tc.load("data/gender_curves.zip")
gender_results = tc.load("data/best_gender_curves.zip")

In [34]:
subdivided_bodies['vertices']['female'] = tc.cat([
    subdivided_bodies['vertices']['female'][:75],
    subdivided_bodies['vertices']['female'][77:],
])
subdivided_bodies['vertices']['female'] = tc.cat([
    subdivided_bodies['vertices']['female'][:17],
    subdivided_bodies['vertices']['female'][19:],
])

In [5]:
column_curves_measures = {}

for idx, gender in enumerate(genders):

    template = subdivided_bodies['faces'][gender].to(device)

    column_curves_measures[gender] = []
    best_neck = gender_curves[gender][curve_index['neck_girth']][gender_results[gender]['best']['neck_girth']]
    best_neck = tc.FloatTensor(best_neck).to(device)

    best_waist = gender_curves[gender][curve_index['waist_girth']][gender_results[gender]['best']['waist_girth']]
    best_waist = tc.FloatTensor(best_waist).to(device)

    bodies_interator = tqdm(subdivided_bodies['vertices'][gender])
    for body in bodies_interator:

        best_neck_pos = CurveUtils.generate_positions(best_neck, body.to(device))
        best_waist_pos = CurveUtils.generate_positions(best_waist, body.to(device))

        neck_ht = best_neck_pos[:,1].max()
        waist_ht = best_waist_pos[:,1].max()

        neck_width = abs(best_neck_pos[:,2]).max()
        waist_width = abs(best_waist_pos[:,2]).max()

        
        width = (neck_width + (waist_width/2))*1.2
        height = neck_ht - waist_ht
        center = (0, waist_ht + (height/2), -width*2/3)
        column_box = BBox(length=width, width=width, height=height, center=tc.FloatTensor(center).to(device))
        column_curves = CurveGenerator.calculate_curve(body.to(device), column_box, 0, template, device)      


        all_col_curves = []
        for curve in column_curves[0]:
            positions = CurveUtils.generate_positions(tc.FloatTensor(curve).to(device), body.to(device))
            all_col_curves.append(positions[:,0].abs().mean())
        idmin = tc.FloatTensor(all_col_curves).abs().argmin()
    #     save_obj("body.obj", body, template+1)
    #     save_obj("neckwaist.obj", tc.FloatTensor(column_curves[2][idmin]))
    #     column_box.save_limits('bb.obj')
    #     break
    # break
        measure = CurveUtils.linear_distance(tc.FloatTensor(column_curves[2][idmin]))
        
        column_curves_measures[gender].append(measure)

100%|██████████| 40/40 [07:18<00:00, 10.96s/it]


In [35]:
column_curves_measures['male'] = tc.FloatTensor(column_curves_measures['male'])
column_curves_measures['female'] = tc.FloatTensor(column_curves_measures['female'])

In [36]:
measures = pd.read_pickle(f'data/ss20_cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "SS20"]
measures.replace("female ", "female", inplace=True)

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [37]:
errors = abs(column_curves_measures['male'] - (mfd_gender_measures['male']['neck_waist']/10))
gender_results['male'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

errors = abs(column_curves_measures['female'] - (mfd_gender_measures['female']['neck_waist']/10))
gender_results['female'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

In [38]:
(gender_results['male']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,1870.0,0.51,46.89,11.82,10.46
waist_girth,3350.0,0.48,16.50,5.69,4.00
hip_girth,1320.0,0.07,12.82,4.52,3.37
thigh_girth,5430.0,0.38,19.92,6.90,5.09
upper_arm_girth,3900.0,0.03,9.04,3.13,2.05
neck_girth,52680.0,0.17,23.89,6.93,5.74
neck_waist,0.0,1.57,107.24,32.15,22.97


In [39]:
(gender_results['female']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,1470.0,0.21,41.67,17.48,12.65
waist_girth,3010.0,0.07,35.88,10.80,9.54
hip_girth,1090.0,0.65,49.38,11.92,10.88
thigh_girth,4720.0,0.57,35.87,8.27,7.42
upper_arm_girth,3460.0,0.15,13.50,5.42,3.58
neck_girth,38000.0,0.03,12.86,4.94,3.40
neck_waist,0.0,0.01,104.15,22.72,20.48


In [40]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='\n')
    
    gender_measures[gender] = []
    
    for curve_name in our_semantic[:-2]:

        
        index = curve_index[curve_name]
        best_id = gender_results[gender].loc[curve_name]['best']
        best_curve = gender_curves[gender][index][best_id]

        body_measures = []
        for body in subdivided_bodies['vertices'][gender]:
            position = CurveUtils.generate_positions(tc.FloatTensor(best_curve).to(device), body.to(device))
            calculated_measures = CurveUtils.calculate_distances(position)
            body_measures.append(calculated_measures)

        gender_measures[gender].append(body_measures)
        
    gender_measures[gender] = tc.FloatTensor(gender_measures[gender])

tc.save(gender_measures, "data/calculated_measures.zip")

MEASURING FEMALE BODIES
MEASURING MALE BODIES


In [41]:
gender_measures['female'].shape, subdivided_bodies['vertices']['female'].shape

(torch.Size([6, 78]), torch.Size([78, 440834, 3]))

In [47]:
best_gender_measures = {}

for idx, gender in enumerate(genders):

    nbodies = len(subdivided_bodies['vertices'][gender])

    best_gender_measures[gender] = tc.row_stack([
        gender_measures[gender], column_curves_measures[gender]
    ]).numpy()
    best_gender_measures[gender] = list(best_gender_measures[gender]*10)
    

    best_gender_measures[gender].append(tc.arange(2).repeat(nbodies//2).numpy()+1)
    best_gender_measures[gender].append([gender]*nbodies)
    best_gender_measures[gender].append(["our"]*nbodies)
    best_gender_measures[gender].append(mfd_gender_measures[gender].index)

In [48]:
aditional_semantic = ['repetition', "gender", "measures_station", 'subject']
our_measures = pd.concat([
    pd.DataFrame(best_gender_measures['female'], index=our_semantic[:-1]+aditional_semantic).T,
    pd.DataFrame(best_gender_measures['male'], index=our_semantic[:-1]+aditional_semantic).T
])

our_measures.to_pickle("data/ss20_our_measures.zip")

In [49]:
our_measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,gender,measures_station,subject
0,993.067749,821.37207,1055.509766,670.890625,326.88324,327.31781,417.976624,1,female,our,IEEEP1_05
1,995.068481,811.080078,1057.318726,677.070801,328.250366,323.928345,410.531036,2,female,our,IEEEP1_05
2,831.730591,673.6474,933.982788,546.396362,264.765472,293.532318,388.651001,1,female,our,IEEEP1_12
3,831.375488,674.062256,930.065186,548.780334,268.68576,294.44812,390.891144,2,female,our,IEEEP1_12
4,951.35321,746.559753,1017.694214,612.421021,294.973755,325.342957,407.239441,1,female,our,IEEEP1_13
...,...,...,...,...,...,...,...,...,...,...,...
35,1015.64624,921.357422,992.517822,549.547058,322.810303,377.230988,434.756165,2,male,our,IEEEP1_79
36,1049.756592,816.259033,985.972351,554.311401,360.12262,403.694336,483.153564,1,male,our,IEEEP1_87
37,1039.332153,808.704163,984.647705,554.602966,378.637848,408.120148,478.285248,2,male,our,IEEEP1_87
38,1222.334595,1095.716675,1120.17627,652.404419,366.434509,459.751709,531.848267,1,male,our,IEEEP1_89
